# THIS IS A JUPYTER NOTEBOOK VERSION OF RT2 ASSIGNMENT

Hi, here is a Jupyter Notebook implementation. 

To start, the necessary libraries must be imported:

In [ ]:
import rospy
from std_msgs.msg import String
from geometry_msgs.msg import Twist
from sensor_msgs.msg import LaserScan
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal, MoveBaseActionGoal, MoveBaseActionFeedback
from actionlib_msgs.msg import GoalID, GoalStatusArray
from nav_msgs.msg import Odometry
from tf.transformations import *

import jupyros as jr
from jupyros import ros3d
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
from IPython.display import display, clear_output

%matplotlib
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
import math
import time
import os

import getch
import sys, tty
sys.path.append("../src/")
import master
import option1
import option2
import option3

Now, lets visualize the robots movement

In [ ]:
v = ros3d.Viewer()
ros_connect = ros3d.ROSConnection(url="ws://localhost:9090") 
tf_client = ros3d.TFClient(ros=ros_connect, fixed_frame='map')
laser_view = ros3d.LaserScan(topic="/scan", ros=ros_connect, tf_client=tf_client) 
map_view = ros3d.OccupancyGrid(topic="/map", ros=ros_connect, tf_client=tf_client, continuous=True) 
path = ros3d.Path(topic="/move_base/NavfnROS/plan", ros=ros_connect, tf_client=tf_client) 
urdf = ros3d.URDFModel(ros=ros_connect, tf_client=tf_client, path=os.environ.get('JUPYROS_ASSETS_URL', 'http://localhost:3000')) 
g = ros3d.GridModel()
v.objects = [g, laser_view, map_view, path, urdf]
v

In [ ]:
## Push the button

button = master.main()
button

Plotting the graphs for laser scan, odometry, and goal status

In [ ]:
### classifying the class to provide plot of robot movement
class Graphs:
    def __init__(self):
        
        self.figure_laser = plt.figure(figsize=(6,6))
        self.axis_laser = plt.subplot(111, polar=True)
        self.axis_laser.set_thetalim(-np.pi/2,np.pi/2)
        self.axis_laser.set_theta_zero_location("N")
        self.line_laser, = self.axis_laser.plot([],[],'bo')
        
        self.figure_odom, self.axis_odom = plt.subplots()
        plt.grid(True)
        self.line_odom, = plt.plot([] ,[], 'bo')
        
        self.reached = False
        self.reached_goal_counter = 0
        self.non_reached_goal_counter = 0
        self.laser  = [] 
        self.position_x = []
        self.position_y = []

        self.figure_target, self.axis_target = plt.subplots()
        plt.grid(True)
        self.status_list = [0, 0]
        self.data = {'Reached Goals': self.reached_goal_counter, 
                     'Unreached Goals':self.non_reached_goal_counter}
        
        self.target = list(self.data.keys())
        self.values = list(self.data.values())
        self.line_target = plt.bar(self.target, self.values, color=("green", "red", "blue"))
        
    ## init the plots to set the        
    def plot_init(self):
        ## laser history
        self.axis_laser.set_title("Laser of the robot")
        self.axis_laser.set_ylabel("Distance to object")
        
        ### setting the general plot (x, y) 
        self.axis_odom.set_xlim(-15, 15)
        self.axis_odom.set_ylim(-15, 15)
        self.axis_odom.set_title("Position")
        self.axis_odom.set_ylabel("Y-axis")
        self.axis_odom.set_xlabel("X-axis")
        
        ## goal history
        self.axis_target.set_ylim(0, 10)
        return self.line_laser , self.line_odom, self.line_target
    
    ### subscribers callbacks of laser, odom, and goals
    def callback_laser(self, msg):
        global angles
        angles = list(np.arange(msg.angle_min, msg.angle_max+msg.angle_increment, msg.angle_increment))
        self.laser=list(msg.ranges)
        
    def callback_odom(self, msg):
        self.position_y.append(msg.pose.pose.position.y)
        self.position_x.append(msg.pose.pose.position.x)
        
    def callback_goal(self, msg): 
        try: 
            self.status_list.append(msg.status_list[0].status)
            
            ### if the status is reached
            if self.status_list[-1] == 3 and self.status_list[-2] != 3:
                self.reached_goal_counter = self.reached_goal_counter + 1
                
            ### if status is not reached
            elif self.status_list[-1] == 2 and self.status_list[-2] != 2:
                self.non_reached_goal_counter = self.non_reached_goal_counter + 1

        except IndexError:
            pass
    
    ## update plots for laser, odom and goal
    def update_laser(self, frame):
        global angles    
        self.axis_laser.set_rmax(20)
        self.line_laser.set_data(angles,self.laser)
        return self.line_laser
            
    def update_odom(self, frame):
        x = self.position_x
        y = self.position_y
        self.line_odom.set_data(x, y)
        return self.line_odom
    
    def update_goal(self, frame):
        x = self.reached_goal_counter
        y = self.non_reached_goal_counter
        self.updated_data = {'Reached goal': x, 'Non-reached goals': y}
        
        h = list(self.updated_data.values())
        q = list(self.updated_data.keys())
        
        self.line_target = plt.bar(q, h, color=("green", "red"))
        return self.line_target

Here is the plots of them

In [ ]:
plots = Graphs()

### laser scan subscriber animation
sub_laser = rospy.Subscriber("/scan", LaserScan, plots.callback_laser)
animate_laser = FuncAnimation(plots.figure_laser, plots.update_laser, blit = True)

### odometry scan subscriber animation
sub_odom = rospy.Subscriber("/odom", Odometry, plots.callback_odom)
animate_odom = FuncAnimation(plots.figure_odom, plots.update_odom, init_func=plots.plot_init)

### goal status scan subscriber animation
sub_status = rospy.Subscriber("/move_base/status", GoalStatusArray, plots.callback_goal)
animate_target = FuncAnimation(plots.figure_target, plots.update_goal, init_func=plots.plot_init, interval=100)

plt.show()

## 